# Statically calculation of the factor ratio between the user-specified runtime and the suitable Max_trials Metric for Autokera's AutoML solution.

# This Analysis Notebook is structured as Folowing:
- Importing,pre-processing and analysing The metrics-Dataset: This Dataset is based of our last experiences and trails of Autokeras and its also include the actual time needed for Autokeras to complete. and its our main Metric to find a suitable ratio

- Discovering the relationship between our Metrics and Visulize them
- Deleting Any high significant outliers
- Calculating the average Value of the related Metrics and deriving a ratio

## Importing The Dataset

In [54]:
import pandas as pd
import numpy as np
from io import StringIO
import plotly.express as px

In [55]:
# all floating point numbers will be displayed with 2 decimal places.
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

In [56]:
# Read CSV into a DataFrame
data_keras = pd.read_csv("datasets_metrics.csv")
# Filter the Dataframe to only includes Autokeras solutions runs that were seucesfull
data_keras=data_keras[(data_keras["AutoML_solution"] == ":autokeras") & (data_keras["failed"] == 0)] # Failed tryouts are 1 "True"  and not failed are 0 "False"

In [57]:
data_keras.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218 entries, 4 to 2450
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  218 non-null    int64  
 1   AutoML_solution             218 non-null    object 
 2   task                        218 non-null    object 
 3   trainings_id                218 non-null    object 
 4   dataset_name                218 non-null    object 
 5   dataset_size_in_mb          218 non-null    float64
 6   absolute_balanced_accuracy  218 non-null    float64
 7   relative_balanced_accuracy  218 non-null    float64
 8   absolute_rmse               218 non-null    float64
 9   relative_rmse               218 non-null    float64
 10  dataset_rows                218 non-null    int64  
 11  dataset_cols                218 non-null    int64  
 12  runtime                     218 non-null    float64
 13  type                        218 no

In [58]:
data_keras

,Unnamed: 0,AutoML_solution,task,trainings_id,dataset_name,dataset_size_in_mb,absolute_balanced_accuracy,relative_balanced_accuracy,absolute_rmse,relative_rmse,dataset_rows,dataset_cols,runtime,type,missing_values,categorical_features,numerical_features,runtime_limit,failed
4,4,:autokeras,:tabular_classification,64dc93a87e4287dd07d42107,breast-w.csv,0.03,0.95,0.98,9.00,9.00,699,10,4.56,Binary classification,16,1,9,60,0
13,13,:autokeras,:tabular_classification,64dca22c7e4287dd07d42112,credit-approval.csv,0.05,0.78,0.90,9.00,9.00,690,16,4.87,Binary classification,67,10,6,60,0
58,58,:autokeras,:tabular_classification,64dd114bd5d06bd0d8053e1d,profb.csv,0.04,0.59,0.86,9.00,9.00,672,10,5.89,Binary classification,1200,5,5,60,0
67,67,:autokeras,:tabular_classification,64dd251bec8a32a23c1dc94e,pc4.csv,0.27,0.57,0.68,9.00,9.00,1458,38,6.72,Binary classification,0,1,37,60,0
80,80,:autokeras,:tabular_classification,64dd3534373296c8268bc91b,christine.csv,50.24,0.74,0.97,9.00,9.00,5418,1637,153.15,Binary classification,0,38,1599,60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2406,2406,:autokeras,:tabular_regression,657b2d2f691b789c7c33f857,sarcos.csv,13.06,9.00,9.00,2.76,0.00,48933,28,4.19,Regression,0,0,22,24,0
2421,2421,:autokeras,:tabular_regression,657be6f6691b789c7c33f865,sarcos.csv,13.06,9.00,9.00,2.66,0.00,48933,28,9.80,Regression,0,0,22,48,0
2429,2429,:autokeras,:tabular_regression,657c25a9691b789c7c33f86f,physiochemical_protein.csv,3.67,9.00,9.00,5.03,0.00,45730,10,1.12,Regression,0,0,10,3,0
2438,2438,:autokeras,:tabular_regression,657c305a691b789c7c33f879,physiochemical_protein.csv,3.67,9.00,9.00,4.80,0.00,45730,10,1.17,Regression,0,0,10,6,0


In [59]:
data_keras.describe()

,Unnamed: 0,dataset_size_in_mb,absolute_balanced_accuracy,relative_balanced_accuracy,absolute_rmse,relative_rmse,dataset_rows,dataset_cols,runtime,missing_values,categorical_features,numerical_features,runtime_limit,failed
count,218.00,218.00,218.00,218.00,218.00,218.00,218.00,218.00,218.00,218.00,218.00,218.00,218.00,218.00
mean,1229.89,13.61,3.96,4.05,9858.46,5.64,27614.92,195.19,30.08,517.38,5.11,104.55,87.21,0.00
std,692.38,60.80,4.08,4.01,37276.17,4.18,65603.66,635.87,72.77,2770.16,12.87,532.32,91.62,0.00
min,4.00,0.02,0.09,0.11,0.01,0.00,323.00,4.00,0.50,-1.00,-1.00,-1.00,3.00,0.00
25%,673.00,0.10,0.60,0.84,9.00,0.98,846.00,10.00,3.87,0.00,0.00,3.00,12.00,0.00
50%,1232.00,1.35,0.92,0.98,9.00,9.00,6872.00,18.00,8.51,0.00,1.00,10.00,60.00,0.00
75%,1787.50,3.67,9.00,9.00,9.00,9.00,45730.00,49.00,19.50,0.00,3.00,22.00,195.00,0.00
max,2450.00,516.79,9.00,9.00,394108.71,9.00,539383.00,4297.00,693.64,18372.00,62.00,4296.00,240.00,0.00


## Data pre-processing: deleting unwanted columns, Duplicated rows and check for NAN values

### Important NOTE: the **runtime_limit** is renamed to **max_trials** to distinguish between the **runtime given by the user which represent the max_trials** in our data AND **the actual measured runtime which is stored in the "runtime" attribute**

### Dropping The columns that are irrelvant to the analysis

In [60]:
columns_to_drop = ['Unnamed: 0', 'AutoML_solution', 'task', 'trainings_id',"categorical_features","numerical_features"
                   "absolute_balanced_accuracy","relative_balanced_accuracy","absolute_rmse","relative_rmse","failed","missing_values","type","task","absolute_balanced_accuracy"]
data_keras = data_keras.drop(columns=columns_to_drop, errors='ignore')
# rename the runtime_limits to max_trials . in the past experioences runtime_limit=max_trials
data_keras = data_keras.rename(columns={'runtime_limit': 'max_trials'})

### Checking wether The Dataset has NAN values

In [61]:
#check for NaN values
data_keras.isnull().sum().sum()

0

In [62]:
data_keras

,dataset_name,dataset_size_in_mb,dataset_rows,dataset_cols,runtime,numerical_features,max_trials
4,breast-w.csv,0.03,699,10,4.56,9,60
13,credit-approval.csv,0.05,690,16,4.87,6,60
58,profb.csv,0.04,672,10,5.89,5,60
67,pc4.csv,0.27,1458,38,6.72,37,60
80,christine.csv,50.24,5418,1637,153.15,1599,60
...,...,...,...,...,...,...,...
2406,sarcos.csv,13.06,48933,28,4.19,22,24
2421,sarcos.csv,13.06,48933,28,9.80,22,48
2429,physiochemical_protein.csv,3.67,45730,10,1.12,10,3
2438,physiochemical_protein.csv,3.67,45730,10,1.17,10,6


### Plotting the correlation Matrix

In [63]:
import plotly.graph_objects as go
correlation_matrix = data_keras.corr()
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.index,
    colorscale='Viridis',
    colorbar=dict(title='Correlation')
))

# Update layout for better readability
fig.update_layout(title='Correlation Matrix',
                  xaxis=dict(title='Features'),
                  yaxis=dict(title='Features'))

# Show the plot
fig.show()

<ipython-input-63-e2cfead9c6c4>:2: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



### Dropping all the columns that are strongly correlated or not significant for the Analysis

In [64]:
columns_to_drop = ['numerical_features', 'dataset_cols', 'dataset_rows']
data_keras = data_keras.drop(columns=columns_to_drop, errors='ignore')

In [65]:
data_keras

,dataset_name,dataset_size_in_mb,runtime,max_trials
4,breast-w.csv,0.03,4.56,60
13,credit-approval.csv,0.05,4.87,60
58,profb.csv,0.04,5.89,60
67,pc4.csv,0.27,6.72,60
80,christine.csv,50.24,153.15,60
...,...,...,...,...
2406,sarcos.csv,13.06,4.19,24
2421,sarcos.csv,13.06,9.80,48
2429,physiochemical_protein.csv,3.67,1.12,3
2438,physiochemical_protein.csv,3.67,1.17,6


### Detecting douplicates and drop them

#### duplicates indicates to a dataset that been runned on the server multible times with the same maxt_trials value.

#### we make sure to have one max trial for each dataset in order to avoid duplicates that harms our statistical analysis

#### We Use here two options and compare them with each others :

- We take the first row and delete all the rows that has the same dataset name and maxt_trials value

- we take the average runtime for the rows with the same dataset and the same maxt_trials value

In [66]:
result_dup=data_keras.copy()
result_dup.drop_duplicates(subset=["dataset_name","dataset_size_in_mb","max_trials"], keep='first', inplace=True)
result_dup.reset_index(inplace=True)
result_dup
#print(result_dup.to_string())



,index,dataset_name,dataset_size_in_mb,runtime,max_trials
0,4,breast-w.csv,0.03,4.56,60
1,13,credit-approval.csv,0.05,4.87,60
2,58,profb.csv,0.04,5.89,60
3,67,pc4.csv,0.27,6.72,60
4,80,christine.csv,50.24,153.15,60
...,...,...,...,...,...
156,2406,sarcos.csv,13.06,4.19,24
157,2421,sarcos.csv,13.06,9.80,48
158,2429,physiochemical_protein.csv,3.67,1.12,3
159,2438,physiochemical_protein.csv,3.67,1.17,6


In [67]:
result_avg = data_keras.groupby(["dataset_name","dataset_size_in_mb","max_trials"])['runtime'].mean().reset_index()
result_avg
#print(result_avg.to_string())
result_avg

,dataset_name,dataset_size_in_mb,max_trials,runtime
0,Amazon_employee_access.csv,2.73,3,1.04
1,Amazon_employee_access.csv,2.73,60,10.71
2,Amazon_employee_access.csv,2.73,240,31.72
3,Bioresponse_short.csv,2.61,3,29.71
4,Bioresponse_short.csv,2.61,6,40.01
...,...,...,...,...
156,wave_energy.csv,33.77,240,14.53
157,waveform-5000.csv,1.07,60,3.89
158,waveform-5000.csv,1.07,240,6.37
159,white_wine.csv,0.30,60,5.26


#### Comparing Both subsets and see wether the runtime difference is significant

In [68]:
x=result_avg[result_avg.dataset_name == "one-hundred-plants-margin.csv"]
x

,dataset_name,dataset_size_in_mb,max_trials,runtime
93,one-hundred-plants-margin.csv,0.80,3,1.81
94,one-hundred-plants-margin.csv,0.80,6,54.33
95,one-hundred-plants-margin.csv,0.80,12,147.22
96,one-hundred-plants-margin.csv,0.80,24,264.33
97,one-hundred-plants-margin.csv,0.80,48,693.64
98,one-hundred-plants-margin.csv,0.80,60,15.70
99,one-hundred-plants-margin.csv,0.80,240,22.32


In [69]:
result_dup[result_dup.dataset_name=="one-hundred-plants-margin.csv"]


,index,dataset_name,dataset_size_in_mb,runtime,max_trials
22,469,one-hundred-plants-margin.csv,0.80,15.70,60
82,1181,one-hundred-plants-margin.csv,0.80,22.32,240
136,1906,one-hundred-plants-margin.csv,0.80,3.11,6
137,1923,one-hundred-plants-margin.csv,0.80,290.48,12
138,1932,one-hundred-plants-margin.csv,0.80,416.15,24
139,1942,one-hundred-plants-margin.csv,0.80,693.64,48
141,2156,one-hundred-plants-margin.csv,0.80,1.81,3


In [70]:
result_dup["runtime"].sum()

5277.017500000001

In [71]:
result_avg["runtime"].sum()

4801.391667857142

In [72]:
Diff=result_dup["runtime"].sum()-result_avg["runtime"].sum()
Diff



475.6258321428586

#### the difference between taking the average and keeping the first maxtrail is Huge. Thats why we take the average runtime of the dataset that has the same name, size and max_trial

In [73]:
data_keras=result_avg.copy()
data_keras

,dataset_name,dataset_size_in_mb,max_trials,runtime
0,Amazon_employee_access.csv,2.73,3,1.04
1,Amazon_employee_access.csv,2.73,60,10.71
2,Amazon_employee_access.csv,2.73,240,31.72
3,Bioresponse_short.csv,2.61,3,29.71
4,Bioresponse_short.csv,2.61,6,40.01
...,...,...,...,...
156,wave_energy.csv,33.77,240,14.53
157,waveform-5000.csv,1.07,60,3.89
158,waveform-5000.csv,1.07,240,6.37
159,white_wine.csv,0.30,60,5.26


#Visualisation of the attributes

## Create and visulize ratio attributes to discover if any relation between Runtime, Max trial and Data-Size is clear to see

In [74]:
# creat new Columns
data_keras["measured_runtime_tosize"]=data_keras["runtime"]/data_keras["dataset_size_in_mb"]
data_keras["given_maxtrials_tosize"]=data_keras["max_trials"]/data_keras["dataset_size_in_mb"]

In [75]:

fig = px.scatter(data_keras, x='given_maxtrials_tosize', y='runtime',
                 title='Scatter Plot between given_maxtrials_tosize and runtime',
                 labels={'given_maxtrials_tosize': 'Given Max Trials to Size', 'runtime': 'Runtime'})

# Show the plot
fig.show()

In [76]:

fig = px.scatter(data_keras, x='measured_runtime_tosize', y='max_trials',
                 title='Scatter Plot between measured_runtime_tosize and max_trials',
                 labels={'measured_runtime_tosize': 'Measured Runtime to Size', 'max_trials': 'Max Trials'})

# Show the plot
fig.show()

In [77]:
data_keras.describe()

,dataset_size_in_mb,max_trials,runtime,measured_runtime_tosize,given_maxtrials_tosize
count,161.00,161.00,161.00,161.00,161.00
mean,12.73,105.83,29.82,42.61,677.61
std,57.96,97.71,74.80,92.52,1740.01
min,0.02,3.00,0.51,0.03,0.09
25%,0.10,24.00,4.34,2.93,6.21
50%,1.26,60.00,8.63,11.64,61.40
75%,3.67,240.00,19.16,48.48,483.74
max,516.79,240.00,693.64,863.88,14477.89


- Its not possible to derive from the Graphs and the stats any significal ratio or relationship between the three variables

## Exploring the disturbution of the indiviual attributes
 - Histogramms to the deviation of the Data:

In [78]:
# Iterate through all columns
for column_name in data_keras.columns:
    # Create a histogram using Plotly Express for each column
    fig = px.histogram(data_keras, x=column_name, title=f'Histogram of {column_name}')
    # Show the plot
    fig.show()

## Removing the outliers related to the dataset_size_in_mb attribute

In [79]:
fig = px.scatter(data_keras, x=data_keras.index, y="dataset_size_in_mb")
fig.show()

## The plot shows two significant outliers that both have a value over 500 mb and should be removed from the Dataset

In [80]:
data_keras = data_keras[data_keras["dataset_size_in_mb"] < 500]

In [81]:
data_keras.describe()

,dataset_size_in_mb,max_trials,runtime,measured_runtime_tosize,given_maxtrials_tosize
count,159.00,159.00,159.00,159.00,159.00
mean,6.39,105.27,27.75,43.14,686.13
std,12.06,97.67,72.91,92.98,1749.31
min,0.02,3.00,0.51,0.03,0.09
25%,0.10,24.00,4.31,3.55,6.92
50%,1.21,60.00,8.20,12.57,65.35
75%,3.67,240.00,18.29,49.35,488.85
max,55.54,240.00,693.64,863.88,14477.89


# Calculating a suitable Ratio

## Dividing the Dataset into 4 Quantiles and try to figure out ratio that can be used to multiplay with the user given runtime and give us a suitable max_trials value.

In [82]:
data_keras['quantile_category'] = pd.qcut(data_keras["dataset_size_in_mb"], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
data_keras["quantile_category"] = data_keras["quantile_category"].astype('category')
input_columns = ['max_trials', 'runtime']
category_column = 'quantile_category'
# Create a scatter plot using Plotly Express
fig = px.scatter(data_keras, x=input_columns[0], y=input_columns[1], color=category_column,
                 title=f'Scatter Plot of {input_columns[0]} and {input_columns[1]} with {category_column}')

# Show the plot
fig.show()

<ipython-input-82-2cc692c86f31>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-82-2cc692c86f31>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [83]:

# Group by 'quantile_category' and calculate average value for 'runtime' and 'max_trials'
summary_stats = data_keras.groupby('quantile_category').agg({
    'runtime': ['mean'],
    'max_trials': ['mean']
}).reset_index()

# Rename the columns for clarity
summary_stats.columns = ['quantile_category', 'avg_runtime', 'avg_max_trials',]

# Merge the summary statistics back into the original DataFrame
data_keras = pd.merge(data_keras, summary_stats, on='quantile_category', how='left')
unique_summary_stats = summary_stats.drop_duplicates()

# Print the unique summary statistics DataFrame
unique_summary_stats
# Print the updated DataFrame
unique_summary_stats

,quantile_category,avg_runtime,avg_max_trials
0,Q1,4.64,94.93
1,Q2,40.24,133.15
2,Q3,16.47,91.57
3,Q4,52.97,102.89


In [84]:
fig = px.scatter(unique_summary_stats, x='avg_max_trials', y='avg_runtime', color='quantile_category',
                 title='Scatter Plot of avg_runtime and avg_max_trials by quantile_category',
                 labels={'avg_runtime': 'Average Runtime', 'avg_max_trials': 'Average Max Trials'})

# Show the plot
fig.show()

In [85]:
average_values = unique_summary_stats[['avg_runtime', 'avg_max_trials']].mean()
average_row = pd.DataFrame([average_values], columns=['avg_runtime', 'avg_max_trials'])
average_row

,avg_runtime,avg_max_trials
0,28.58,105.64


# This result shows that, on average, every 1 minute of runtime can compute an estimation of 3.7 max_trials for dataset sizes between 0 and 55.54 Megabytes in our Servers.